<a href="https://colab.research.google.com/github/immprasetyo/ibm-data-science/blob/main/spacex-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **SpaceX  Falcon 9 First Stage Landing Prediction**


# Lab 1: Collecting the Data


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. SpaceX performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [1]:
import numpy as np
import pandas as pd
import requests
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)

# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

__Global Variables__<br />
booster_version = []<br />
longitude = []<br />
latitude = []<br />
launch_site = []<br />
payload_mass = []<br />
orbit = []<br />
serial = []<br />
block = []<br />
reused_count = []<br />
reused = []<br />
flights = []<br />
grid_fins = []<br />
legs = []<br />
landing_pad = []<br />
outcome = []

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [2]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def get_booster_version(df):
    for i in df['rocket']:
        if i:
            response = requests.get('https://api.spacexdata.com/v4/rockets/' + str(i)).json()
            booster_version.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [3]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def get_launch_site(df):
    for i in df['launchpad']:
        if i:
            response = requests.get('https://api.spacexdata.com/v4/launchpads/' + str(i)).json()
            longitude.append(response['longitude'])
            latitude.append(response['latitude'])
            launch_site.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [4]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def get_payload_data(df):
    for i in df['payloads']:
        if i:
            response = requests.get('https://api.spacexdata.com/v4/payloads/' + i).json()
            payload_mass.append(response['mass_kg'])
            orbit.append(response['orbit'])

From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [5]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def get_core_data(df):
    for i in df['cores']:
            if i['core'] != None:
                response = requests.get('https://api.spacexdata.com/v4/cores/' + i['core']).json()
                serial.append(response['serial'])
                block.append(response['block'])
                reused_count.append(response['reuse_count'])
            else:
                block.append(None)
                reused_count.append(None)
                serial.append(None)
            reused.append(i['reused'])
            flights.append(i['flight'])
            grid_fins.append(i['gridfins'])
            legs.append(i['legs'])
            landing_pad.append(i['landpad'])
            outcome.append(str(i['landing_success']) + ' ' + str(i['landing_type']))

Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [6]:
spacex_api_url = 'https://api.spacexdata.com/v4/launches/past'

In [7]:
response = requests.get(spacex_api_url)

Check the content of the response


In [ ]:
#response.content

You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


### Task 1: Request and parse the SpaceX launch data using the GET request


We should see that the request was successfull with the 200 status response code


In [8]:
response.status_code

200

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [ ]:
# View the response
#response.json()

In [9]:
# Use json_normalize method to convert the json result into a dataframe
spacex_df = pd.json_normalize(response.json())

Using the dataframe <code>data</code> print the first 5 rows


In [10]:
# Get the head of the dataframe
spacex_df.head()

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/94/f2/NN6Ph45r_o.png,https://images2.imgbox.com/5b/02/QcxHUb5V_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutdown'}]","Successful first stage burn and transition to second stage, maximum altitude 289 km, Premature engine shutdown at T+7 min 30 s, Failed to reach orbit, Failed to recover first stage",[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdaffd86e000604b32b,False,False,False,[],https://images2.imgbox.com/f9/4a/ZboXReNb_o.png,https://images2.imgbox.com/80/a2/bkWotCIS_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-rocket-fails-reach-orbit.html,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'residual stage-1 thrust led to collision between stage 1 and stage 2'}]",Residual stage 1 thrust led to collision between stage 1 and stage 2,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006eeb1e4]",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdbffd86e000604b32c,False,False,False,[],https://images2.imgbox.com/6c/cb/na1tzhHs_o.png,https://images2.imgbox.com/4a/80/k1oAkY0k_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1-flight-3-mission-summary,https://en.wikipedia.org/wiki/Trailblazer_(satellite),NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],"Ratsat was carried to orbit on the first successful orbital launch of any privately funded and developed, liquid-propelled carrier rocket, the SpaceX Falcon 1",[],[],[],[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,4,RatSat,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': 

In [11]:
spacex_df.shape

(187, 43)

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [12]:
# Take a subset of our dataframe keeping only the features we want, including the flight number and date_utc
spacex_df = spacex_df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# Remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket
spacex_df = spacex_df[spacex_df['cores'].map(len) == 1]
spacex_df = spacex_df[spacex_df['payloads'].map(len) == 1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature
spacex_df['cores'] = spacex_df['cores'].map(lambda x: x[0])
spacex_df['payloads'] = spacex_df['payloads'].map(lambda x: x[0])

# Convert the date_utc to a datetime datatype and then extracting the date leaving the time
spacex_df['date'] = pd.to_datetime(spacex_df['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
spacex_df = spacex_df[spacex_df['date'] <= datetime.date(2020, 11, 13)]

* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [13]:
# Global variables
booster_version = []
longitude = []
latitude = []
launch_site = []
payload_mass = []
orbit = []
serial = []
block = []
reused_count = []
reused = []
flights = []
grid_fins = []
legs = []
landing_pad = []
outcome = []

These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


In [14]:
booster_version

[]

Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


In [15]:
# Call get_booster_version
get_booster_version(spacex_df)

the list has now been update 


In [16]:
booster_version[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

we can apply the rest of the  functions here:


In [17]:
# Call get_launch_site
get_launch_site(spacex_df)

In [18]:
# Call get_payload_data
get_payload_data(spacex_df)

In [19]:
# Call get_core_data
get_core_data(spacex_df)

Finally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [20]:
spacex_df.head()

,rocket,payloads,launchpad,cores,flight_number,date_utc,date
0,5e9d0d95eda69955f709d1eb,5eb0e4b5b6c3bb0006eeb1e1,5e9e4502f5090995de566f86,"{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",1,2006-03-24T22:30:00.000Z,2006-03-24
1,5e9d0d95eda69955f709d1eb,5eb0e4b6b6c3bb0006eeb1e2,5e9e4502f5090995de566f86,"{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",2,2007-03-21T01:10:00.000Z,2007-03-21
3,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e5,5e9e4502f5090995de566f86,"{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",4,2008-09-28T23:15:00.000Z,2008-09-28
4,5e9d0d95eda69955f709d1eb,5eb0e4b7b6c3bb0006eeb1e6,5e9e4502f5090995de566f86,"{'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",5,2009-07-13T03:35:00.000Z,2009-07-13
5,5e9d0d95eda69973a809d1ec,5eb0e4b7b6c3bb0006eeb1e7,5e9e4501f509094ba4566f84,"{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",6,2010-06-04T18:45:00.000Z,2010-06-04


In [21]:
spacex_df.shape

(94, 7)

In [22]:
print(len(booster_version))
print(len(outcome))

94
94


In [23]:
spacex_dict = {
    'Flight Number': list(spacex_df['flight_number']),
    'Date': list(spacex_df['date']),
    'Booster Version': booster_version,
    'Longitude': longitude,
    'Latitude': latitude,
    'Launch Site': launch_site,
    'Payload Mass': payload_mass,
    'Orbit': orbit,
    'Serial': serial,
    'Block': block,
    'Reused Count': reused_count,
    'Reused': reused,
    'Flights': flights,
    'Grid Fins': grid_fins,
    'Legs': legs,
    'Landing Pad': landing_pad,
    'Outcome': outcome
}

Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [24]:
# Create a data from spacex_dict
spacex_df = pd.DataFrame(spacex_dict)

Show the summary of the dataframe


In [25]:
# Head of the dataframe
spacex_df.head()

,Flight Number,Date,Booster Version,Longitude,Latitude,Launch Site,Payload Mass,Orbit,Serial,Block,Reused Count,Reused,Flights,Grid Fins,Legs,Landing Pad,Outcome
0,1,2006-03-24,Falcon 1,167.743129,9.047721,Kwajalein Atoll,20.0,LEO,Merlin1A,NaN,0,False,1,False,False,None,None None
1,2,2007-03-21,Falcon 1,167.743129,9.047721,Kwajalein Atoll,NaN,LEO,Merlin2A,NaN,0,False,1,False,False,None,None None
2,4,2008-09-28,Falcon 1,167.743129,9.047721,Kwajalein Atoll,165.0,LEO,Merlin2C,NaN,0,False,1,False,False,None,None None
3,5,2009-07-13,Falcon 1,167.743129,9.047721,Kwajalein Atoll,200.0,LEO,Merlin3C,NaN,0,False,1,False,False,None,None None
4,6,2010-06-04,Falcon 9,-80.577366,28.561857,CCSFS SLC 40,NaN,LEO,B0003,1.0,0,False,1,False,False,None,None None


In [26]:
spacex_df.shape

(94, 17)

### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [27]:
# Filter the dataframe to only include Falcon 9 launches
spacex_df = spacex_df[spacex_df['Booster Version'] == 'Falcon 9']
spacex_df.shape

(90, 17)

Now that we have removed some values we should reset the FlgihtNumber column


In [28]:
spacex_df.loc[:, 'Flight Number'] = list(range(1, spacex_df.shape[0] + 1))
spacex_df

,Flight Number,Date,Booster Version,Longitude,Latitude,Launch Site,Payload Mass,Orbit,Serial,Block,Reused Count,Reused,Flights,Grid Fins,Legs,Landing Pad,Outcome
4,1,2010-06-04,Falcon 9,-80.577366,28.561857,CCSFS SLC 40,NaN,LEO,B0003,1.0,0,False,1,False,False,None,None None
5,2,2012-05-22,Falcon 9,-80.577366,28.561857,CCSFS SLC 40,525.0,LEO,B0005,1.0,0,False,1,False,False,None,None None
6,3,2013-03-01,Falcon 9,-80.577366,28.561857,CCSFS SLC 40,677.0,ISS,B0007,1.0,0,False,1,False,False,None,None None
7,4,2013-09-29,Falcon 9,-120.610829,34.632093,VAFB SLC 4E,500.0,PO,B1003,1.0,0,False,1,False,False,None,False Ocean
8,5,2013-12-03,Falcon 9,-80.577366,28.561857,CCSFS SLC 40,3170.0,GTO,B1004,1.0,0,False,1,False,False,None,None None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,86,2020-09-03,Falcon 9,-80.603956,28.608058,KSC LC 39A,15600.0,VLEO,B1060,5.0,12,True,2,True,True,5e9e3032383ecb6bb234e7ca,True ASDS
90,87,2020-10-06,Falcon 9,-80.603956,28.608058,KSC LC 39A,15600.0,VLEO,B1058,5.0,13,True,3,True,True,5e9e3032383ecb6bb234e7ca,True ASDS
91,88,2020-10-18,Falcon 9,-80.603956,28.608058,KSC LC 39A,15600.0,VLEO,B1051,5.0,12,True,6,True,True,5e9e3032383ecb6bb234e7ca,True ASDS
92,89,2020-10-24,Falcon 9,-80.577366,28.561857,CCSFS SLC 40,15600.0,VLEO,B1060,5.0,12,True,3,True,True,5e9e3033383ecbb9e534e7cc,True ASDS


In [30]:
spacex_df.reset_index(drop = True, inplace = True)
spacex_df

,Flight Number,Date,Booster Version,Longitude,Latitude,Launch Site,Payload Mass,Orbit,Serial,Block,Reused Count,Reused,Flights,Grid Fins,Legs,Landing Pad,Outcome
0,1,2010-06-04,Falcon 9,-80.577366,28.561857,CCSFS SLC 40,NaN,LEO,B0003,1.0,0,False,1,False,False,None,None None
1,2,2012-05-22,Falcon 9,-80.577366,28.561857,CCSFS SLC 40,525.0,LEO,B0005,1.0,0,False,1,False,False,None,None None
2,3,2013-03-01,Falcon 9,-80.577366,28.561857,CCSFS SLC 40,677.0,ISS,B0007,1.0,0,False,1,False,False,None,None None
3,4,2013-09-29,Falcon 9,-120.610829,34.632093,VAFB SLC 4E,500.0,PO,B1003,1.0,0,False,1,False,False,None,False Ocean
4,5,2013-12-03,Falcon 9,-80.577366,28.561857,CCSFS SLC 40,3170.0,GTO,B1004,1.0,0,False,1,False,False,None,None None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,86,2020-09-03,Falcon 9,-80.603956,28.608058,KSC LC 39A,15600.0,VLEO,B1060,5.0,12,True,2,True,True,5e9e3032383ecb6bb234e7ca,True ASDS
86,87,2020-10-06,Falcon 9,-80.603956,28.608058,KSC LC 39A,15600.0,VLEO,B1058,5.0,13,True,3,True,True,5e9e3032383ecb6bb234e7ca,True ASDS
87,88,2020-10-18,Falcon 9,-80.603956,28.608058,KSC LC 39A,15600.0,VLEO,B1051,5.0,12,True,6,True,True,5e9e3032383ecb6bb234e7ca,True ASDS
88,89,2020-10-24,Falcon 9,-80.577366,28.561857,CCSFS SLC 40,15600.0,VLEO,B1060,5.0,12,True,3,True,True,5e9e3033383ecbb9e534e7cc,True ASDS


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [31]:
spacex_df.isnull().sum()

Flight Number       0
Date                0
Booster Version     0
Longitude           0
Latitude            0
Launch Site         0
Payload Mass        5
Orbit               0
Serial              0
Block               0
Reused Count        0
Reused              0
Flights             0
Grid Fins           0
Legs                0
Landing Pad        26
Outcome             0
dtype: int64

Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with missing values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [32]:
# Calculate the mean value of Payload Mass column
mean = spacex_df['Payload Mass'].mean()
mean

6123.547647058824

In [34]:
# Replace the np.nan values with its mean value
spacex_df['Payload Mass'].fillna(mean, inplace = True)
spacex_df.isnull().sum()

Flight Number       0
Date                0
Booster Version     0
Longitude           0
Latitude            0
Launch Site         0
Payload Mass        0
Orbit               0
Serial              0
Block               0
Reused Count        0
Reused              0
Flights             0
Grid Fins           0
Legs                0
Landing Pad        26
Outcome             0
dtype: int64

You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


In [ ]:
spacex_df.to_csv('spacex_api_df.csv', index = False)

## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|


Copyright © 2021 IBM Corporation. All rights reserved.
